In [2]:
!unzip "/content/drive/MyDrive/iit Hydrabad/adversarial_50.zip" -d /content/adversarial_50

Archive:  /content/drive/MyDrive/iit Hydrabad/adversarial_50.zip
   creating: /content/adversarial_50/content/adversarial_50/
   creating: /content/adversarial_50/content/adversarial_50/adversarial/
   creating: /content/adversarial_50/content/adversarial_50/adversarial/images/
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_399.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_123.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_054.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_409.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_129.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_481.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversarial/images/adv_452.png  
  inflating: /content/adversarial_50/content/adversarial_50/adversaria

In [3]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.

In [21]:
import os
from collections import OrderedDict
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image
import flwr as fl
from flwr.client import NumPyClient, Client, ClientApp
from flwr.common import Metrics, NDArrays, Config, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLIENTS = 3   # total number of clients (2 clean + 1 malicious)
BATCH_SIZE = 4    # adjust as needed
NUM_ROUNDS = 5    # federated rounds

print(f"Running on device: {DEVICE}")
print(f"Flower version: {fl.__version__}")


Running on device: cuda
Flower version: 1.17.0


In [22]:
NUM_CLIENTS = 3
BATCH_SIZE = 4
NUM_ROUNDS = 1  # Number of federated training rounds


In [7]:
class CustomSegDataset(Dataset):
    """Dataset for (image, mask) pairs, resized to 512×512."""
    def __init__(self, image_paths: List[str], mask_paths: List[str], transform=None):
        super().__init__()
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # single-channel

        if self.transform is not None:
            # We'll do the same 512×512 resize for both image and mask.
            image = self.transform(image)

            # For the mask, do a manual resize + ToTensor if needed:
            mask = transforms.functional.resize(mask, (512, 512))
            mask = transforms.functional.to_tensor(mask)  # shape [1,H,W]

        # For a 2-class segmentation, ensure mask is long dtype
        mask = mask.long().squeeze(dim=0)  # shape [H,W]

        return image, mask


In [49]:
from torch.utils.data import random_split
from typing import Tuple, List ,Dict,Union,Optional

def load_data_for_client(part_id: int) -> Tuple[DataLoader, DataLoader]:
    """
    Creates training and validation DataLoaders for a given client partition.
    - For part_id == 0: first 250 clean images.
    - For part_id == 1: next 250 clean images.
    - For part_id == 2 (malicious): 250 clean + 250 adversarial images.
    The dataset is split 80% for training and 20% for validation.
    """
    base_dir = "/content/adversarial_50/content/adversarial_50"  # Adjust if needed

    # Directories for clean data
    clean_imgs_dir = os.path.join(base_dir, "clean", "images")
    clean_msks_dir = os.path.join(base_dir, "clean", "masks")
    all_clean_imgs = sorted([os.path.join(clean_imgs_dir, f)
                             for f in os.listdir(clean_imgs_dir) if f.endswith(".png")])
    all_clean_msks = sorted([os.path.join(clean_msks_dir, f)
                             for f in os.listdir(clean_msks_dir) if f.endswith(".png")])

    # Directories for adversarial data
    adv_imgs_dir = os.path.join(base_dir, "adversarial", "images")
    adv_msks_dir = os.path.join(base_dir, "adversarial", "masks")
    all_adv_imgs = sorted([os.path.join(adv_imgs_dir, f)
                           for f in os.listdir(adv_imgs_dir) if f.endswith(".png")])
    all_adv_msks = sorted([os.path.join(adv_msks_dir, f)
                           for f in os.listdir(adv_msks_dir) if f.endswith(".png")])

    # Partitioning logic:
    if part_id == 0:
        imgs = all_clean_imgs[0:166]
        msks = all_clean_msks[0:166]
    elif part_id == 1:
        imgs = all_clean_imgs[166:332]
        msks = all_clean_msks[166:332]
    else:
        # Malicious client: combine 250 clean and 250 adversarial images
        imgs = all_clean_imgs[332:500] + all_adv_imgs[0:166]
        msks = all_clean_msks[332:500] + all_adv_msks[0:166]

    # Transformation: resize to 512x512 and convert to tensor.
    transform_img = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])

    # Build full dataset for this partition
    dataset = CustomSegDataset(imgs, msks, transform=transform_img)
    total = len(dataset)
    train_size = int(0.8 * total)
    val_size = total - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    return train_loader, val_loader


In [9]:
def create_deeplabv3_model(num_classes=2) -> nn.Module:
    model = torchvision.models.segmentation.deeplabv3_resnet50(weights=None)
    # Adjust the final classifier
    model.classifier[-1] = nn.Conv2d(256, num_classes, kernel_size=1)
    return model.to(DEVICE)

In [10]:
def dice_score(pred_mask: torch.Tensor, true_mask: torch.Tensor, smooth=1.0) -> float:
    """Compute Dice for binary segmentation. Adapt if multiple classes."""
    pred_flat = pred_mask.view(-1).float()
    true_flat = true_mask.view(-1).float()
    intersection = (pred_flat * true_flat).sum()
    dice = (2.0 * intersection + smooth) / (pred_flat.sum() + true_flat.sum() + smooth)
    return dice.item()


def train_one_epoch(model: nn.Module, loader: DataLoader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for images, masks in loader:
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)["out"]  # shape [N,2,H,W]
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

In [11]:
def evaluate_dice(model: nn.Module, loader: DataLoader) -> float:
    """Evaluate model on given DataLoader using Dice score."""
    model.eval()
    all_dice = []
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)["out"]
            preds = outputs.argmax(dim=1)  # [N,H,W]
            for i in range(preds.shape[0]):
                d = dice_score(preds[i], masks[i])
                all_dice.append(d)
    return float(np.mean(all_dice)) if all_dice else 0.0

In [12]:
def get_parameters(model: nn.Module) -> List[np.ndarray]:
    return [p.cpu().numpy() for p in model.state_dict().values()]

def set_parameters(model: nn.Module, parameters: List[np.ndarray]):
    state_dict = OrderedDict()
    for (k, v), new_w in zip(model.state_dict().items(), parameters):
        # Ensure matching dtype/device
        state_dict[k] = torch.tensor(new_w, dtype=v.dtype, device=DEVICE)
    model.load_state_dict(state_dict, strict=True)

In [31]:
class SegmentationClient(NumPyClient):
    def __init__(self, cid: int, model: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs=1):
        super().__init__()
        self.cid = cid
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        self.epochs = epochs

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        optimizer = optim.Adam(self.model.parameters(), lr=1e-4)
        criterion = nn.CrossEntropyLoss()

        for e in range(self.epochs):
            train_loss = train_one_epoch(self.model, self.trainloader, optimizer, criterion)
            print(f"[Client {self.cid}] Epoch {e+1} train loss: {train_loss:.4f}")
        return get_parameters(self.model), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        self.model.eval()
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        total_samples = 0
        dice_vals = []
        with torch.no_grad():
            for images, masks in self.valloader:
                images, masks = images.to(DEVICE), masks.to(DEVICE)
                outputs = self.model(images)["out"]  # shape [N, num_classes, H, W]
                loss = criterion(outputs, masks)
                batch_size = images.size(0)
                total_loss += loss.item() * batch_size
                total_samples += batch_size
                preds = outputs.argmax(dim=1)
                for i in range(batch_size):
                    d = dice_score(preds[i], masks[i])
                    dice_vals.append(d)
        avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
        avg_dice = float(np.mean(dice_vals)) if dice_vals else 0.0
        print(f"[Client {self.cid}] Local evaluation loss: {avg_loss:.4f}, Dice: {avg_dice:.4f}")
        return float(avg_loss), total_samples, {"dice": avg_dice}

In [44]:
def load_clean_eval_data() -> DataLoader:
    """
    For demonstration, suppose we re-use the last 10 'clean' images for global eval.
    Adjust as you want.
    """
    base_dir = "/content/adversarial_50/content/adversarial_50/"
    clean_imgs_dir = os.path.join(base_dir, "clean", "images")
    clean_msks_dir = os.path.join(base_dir, "clean", "masks")

    # Suppose we only want "clean_041.png" through "clean_050.png"
    eval_imgs = sorted([
        os.path.join(clean_imgs_dir, f)
        for f in os.listdir(clean_imgs_dir)
        if f.endswith(".png") and ("041" in f or "042" in f or "043" in f or
                                   "044" in f or "045" in f or "046" in f or
                                   "047" in f or "048" in f or "049" in f or
                                   "050" in f)
    ])
    eval_msks = sorted([
        os.path.join(clean_msks_dir, f)
        for f in os.listdir(clean_msks_dir)
        if f.endswith(".png") and ("041" in f or "042" in f or "043" in f or
                                   "044" in f or "045" in f or "046" in f or
                                   "047" in f or "048" in f or "049" in f or
                                   "050" in f)
    ])

    transform_img = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])

    dataset = CustomSegDataset(eval_imgs, eval_msks, transform=transform_img)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,drop_last=False)
    return loader


class DiceEvalFedAvg(FedAvg):
    def __init__(self, global_model: nn.Module, clean_eval_loader: DataLoader, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.global_model = global_model
        self.clean_eval_loader = clean_eval_loader
        self.latest_parameters = None  # initialize

    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: list[Union[tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes], BaseException]],
    ) -> tuple[Optional[fl.common.Parameters], dict[str, float]]:
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        # Save the aggregated parameters for use in evaluation.
        self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(self, rnd, results, failures):
        aggregated_loss, aggregated_metrics = super().aggregate_evaluate(rnd, results, failures)
        if self.latest_parameters is not None:
            new_params = fl.common.parameters_to_ndarrays(self.latest_parameters)
            set_parameters(self.global_model, new_params)
            dice_val = evaluate_dice(self.global_model, self.clean_eval_loader)
            print(f"[Round {rnd}] Global Dice (on clean eval) = {dice_val:.4f}")
            if aggregated_metrics is not None:
                aggregated_metrics["global_clean_dice"] = dice_val
        return aggregated_loss, aggregated_metrics


In [51]:
def server_fn(ctx: Context) -> ServerAppComponents:
    model = create_deeplabv3_model(num_classes=2)
    clean_eval_loader = load_clean_eval_data()

    strategy = DiceEvalFedAvg(
        global_model=model,
        clean_eval_loader=clean_eval_loader,
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=NUM_CLIENTS,
        min_evaluate_clients=NUM_CLIENTS,
        min_available_clients=NUM_CLIENTS,
        evaluate_metrics_aggregation_fn=weighted_average,
    )
    config = ServerConfig(num_rounds=NUM_ROUNDS)
    return ServerAppComponents(strategy=strategy, config=config)


def client_fn(context: Context) -> Client:
    # "partition-id" is provided via node_configs.
    partition_id = context.node_config["partition-id"]
    trainloader, valloader = load_data_for_client(partition_id)
    local_model = create_deeplabv3_model(num_classes=2)
    return SegmentationClient(partition_id, local_model, trainloader, valloader, epochs=5).to_client()

In [47]:
def weighted_average(metrics: List[Tuple[int, Dict[str, float]]]) -> Dict[str, float]:
    total_samples = sum(num for num, _ in metrics)
    aggregated_dice = sum(num * m.get("dice", 0.0) for num, m in metrics)
    return {"global_clean_dice": aggregated_dice / total_samples if total_samples > 0 else 0.0}

In [52]:
if __name__ == "__main__":
    backend_config = {
        "client_resources": {"num_cpus": 1, "num_gpus": 0.0}
    }
    if DEVICE == "cuda":
        backend_config["client_resources"]["num_gpus"] = 1.0

    # Provide node_configs so that each of the 3 clients is assigned a partition-id
    node_configs = [
        {"partition-id": 0},  # client 0
        {"partition-id": 1},  # client 1
        {"partition-id": 2},  # client 2 (malicious)
    ]

    clean_eval_loader = load_clean_eval_data()

    # Define your global model
    global_model = create_deeplabv3_model(num_classes=2)

    # Create an instance of the DiceEvalFedAvg strategy
    strategy = DiceEvalFedAvg(
        global_model=global_model,  # Pass the global model
        clean_eval_loader=clean_eval_loader,  # Pass the clean evaluation loader
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=NUM_CLIENTS,
        min_evaluate_clients=NUM_CLIENTS,
        min_available_clients=NUM_CLIENTS,
        evaluate_metrics_aggregation_fn=weighted_average,
    )
    # Modify the server_fn to use the strategy we created:
    def server_fn_modified(ctx: Context) -> ServerAppComponents:
        config = ServerConfig(num_rounds=NUM_ROUNDS)
        return ServerAppComponents(strategy=strategy, config=config)

    # Create the server app using the modified server_fn:
    server_app = ServerApp(server_fn=server_fn_modified)

    fl.simulation.run_simulation(
        server_app=server_app,
    client_app=ClientApp(client_fn=client_fn),
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
    )

INFO :      Starting Flower ServerApp, config: num_rounds=1, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=19708) 2025-03-25 16:08:52.491419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=19708) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=19708) E0000 00:00:1742918932.513757   19708 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=19708) E0000 00:00:1742918932.520431   19708 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INF

(ClientAppActor pid=19708) [Client 0] Epoch 1 train loss: 0.3932
(ClientAppActor pid=19708) [Client 0] Epoch 2 train loss: 0.1053
(ClientAppActor pid=19708) [Client 0] Epoch 3 train loss: 0.0585
(ClientAppActor pid=19708) [Client 0] Epoch 4 train loss: 0.0440
(ClientAppActor pid=19708) [Client 0] Epoch 5 train loss: 0.0366
(ClientAppActor pid=19708) [Client 1] Epoch 1 train loss: 0.3833
(ClientAppActor pid=19708) [Client 1] Epoch 2 train loss: 0.0969
(ClientAppActor pid=19708) [Client 1] Epoch 3 train loss: 0.0602
(ClientAppActor pid=19708) [Client 1] Epoch 4 train loss: 0.0489
(ClientAppActor pid=19708) [Client 1] Epoch 5 train loss: 0.0374
(ClientAppActor pid=19708) [Client 2] Epoch 1 train loss: 0.2297
(ClientAppActor pid=19708) [Client 2] Epoch 2 train loss: 0.0688
(ClientAppActor pid=19708) [Client 2] Epoch 3 train loss: 0.0483
(ClientAppActor pid=19708) [Client 2] Epoch 4 train loss: 0.0392
(ClientAppActor pid=19708) [Client 2] Epoch 5 train loss: 0.0336


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=19708) [Client 0] Local evaluation loss: 0.1722, Dice: 0.4531
(ClientAppActor pid=19708) [Client 1] Local evaluation loss: 0.1727, Dice: 0.5406


INFO :      aggregate_evaluate: received 3 results and 0 failures


(ClientAppActor pid=19708) [Client 2] Local evaluation loss: 0.1723, Dice: 0.2843


INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 1 round(s) in 912.00s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.17236731659483026
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'global_clean_dice': [(1, 0.6816783913644031)]}
INFO :      


[Round 1] Global Dice (on clean eval) = 0.6817
